In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from matplotlib import rcParams
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# figure size in inches
rcParams['figure.figsize'] = 11.7,8.27
from keras.utils import to_categorical
import keras
from sklearn.metrics import accuracy_score
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, Add
from keras.layers import Input, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
from keras import regularizers
from keras import layers
from keras.models import Model
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
root="/content/drive/My Drive/dataset/dataset/"

In [4]:
def image_resize(image_array):
  resized_image=cv2.resize(image_array, (224,224))
  return resized_image

In [5]:
def image_to_array(name):
  image = cv2.imread(root+ '/train/' + name)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  #plt.imshow(si)
  return image

In [6]:
def prepro(root):
  df = pd.read_csv(root+'train.csv')
  y_list =[]
  dictionary = { 
    'manipuri' :0,
    'bharatanatyam':1,
    'odissi' :2,
    'kathakali' : 3,
    'kathak' :4,
    'sattriya':5,
    'kuchipudi':6,
    'mohiniyattam':7
}

  for i in df['target']:
    y_list.append(dictionary[i])
  y=np.array(y_list)
  one_hot = to_categorical(y, num_classes=8)
  image_list=[]
  for i in df['Image']:
    image = image_to_array(i)
    image = image_resize(image)
    image_list.append(image)
  X=np.array(image_list)
  return X,one_hot

In [7]:
X,y = prepro(root)

In [8]:
y

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [9]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [19]:
X_train, X_val, y_train, y_val = train_test_split( X, y, test_size=0.1)

In [20]:
image_gen = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

#training the image preprocessing
image_gen.fit(X_train, augment=True)

In [21]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [22]:
def train_split(X,y,ratio):
  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)
  y_train = to_categorical(y=y_train, num_classes=8)
  y_val = to_categorical(y=y_val, num_classes=8)
  return (X_train,y_train,X_val,y_val)

In [23]:
def Model(n,drop_prob):
  pretrained_model=keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=(224,224,3))
  pretrained_model.trainable=False
  #pretrained_model = modify(pretrained_model,k)
  model = Sequential([
  pretrained_model,
  keras.layers.Flatten(),
  keras.layers.Dropout(drop_prob),
  keras.layers.Dense(8, activation='softmax')])
  return model
    # include_top false to not include  final layers
    #model = keras.applications.InceptionV3(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

In [24]:
def predict(data,model,batchsize,learning_rate,loss,epoc):

  (X_train, X_val, y_train, y_val) = data
 # opt = keras.optimizers.Adam(learning_rate)
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                                                            initial_learning_rate=0.01,
                                                            decay_steps=100000,
                                                            decay_rate=0.7)
  opt = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
  model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=[get_f1])
  model.fit_generator( image_gen.flow(X_train, y_train, batch_size=batchsize),
          steps_per_epoch = 5,
          epochs=epoc,
          validation_data=(X_val,y_val),
          )
  return model

In [25]:
print(X_train.shape,X_val.shape,y_train.shape,y_val.shape)

(327, 224, 224, 3) (37, 224, 224, 3) (327, 8) (37, 8)


VGG 16

In [26]:
model = Model(0,0.1)
#model = modify(model)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 25088)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 200712    
Total params: 14,915,400
Trainable params: 200,712
Non-trainable params: 14,714,688
_________________________________________________________________


In [27]:
loss='categorical_crossentropy'
data=(X_train, X_val, y_train, y_val)
predict(data,model,25,0.01,loss,100)

Epoch 1/100
5/5 [==============================] - 3s 518ms/step - loss: 857.1663 - get_f1: 0.2160 - val_loss: 2179.9780 - val_get_f1: 0.1633
Epoch 2/100
5/5 [==============================] - 2s 322ms/step - loss: 1782.3116 - get_f1: 0.1520 - val_loss: 1058.1507 - val_get_f1: 0.2667
Epoch 3/100
5/5 [==============================] - 2s 393ms/step - loss: 1304.2172 - get_f1: 0.2160 - val_loss: 1342.9018 - val_get_f1: 0.4100
Epoch 4/100
5/5 [==============================] - 2s 394ms/step - loss: 860.8589 - get_f1: 0.4320 - val_loss: 715.5475 - val_get_f1: 0.4483
Epoch 5/100
5/5 [==============================] - 2s 333ms/step - loss: 533.6446 - get_f1: 0.5360 - val_loss: 580.8837 - val_get_f1: 0.4700
Epoch 6/100
5/5 [==============================] - 2s 407ms/step - loss: 423.8447 - get_f1: 0.4480 - val_loss: 348.1407 - val_get_f1: 0.5533
Epoch 7/100
5/5 [==============================] - 2s 417ms/step - loss: 277.4841 - get_f1: 0.6240 - val_loss: 518.8507 - val_get_f1: 0.5000
Epoch 8/

In [93]:
import os

In [130]:
def load_images_from_folder(folder,df):
    images = []
    for filename in df['Image']:
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)   #converting to HSV color space, V is measure of brightness
        print(filename)
        if img is not None:
            images.append(img)
    return images

In [131]:
df_test=pd.read_csv('/content/drive/My Drive/dataset/dataset/test.csv')

In [132]:
X_test=load_images_from_folder('/content/drive/My Drive/dataset/dataset/test',df_test)

508.jpg
246.jpg
473.jpg
485.jpg
128.jpg
410.jpg
465.jpg
196.jpg
340.jpg
467.jpg
306.jpg
171.jpg
323.jpg
332.jpg
330.jpg
170.jpg
479.jpg
9.jpg
283.jpg
114.jpg
507.jpg
183.jpg
200.jpg
457.jpg
264.jpg
194.jpg
518.jpg
175.jpg
64.jpg
152.jpg
270.jpg
125.jpg
38.jpg
440.jpg
302.jpg
394.jpg
448.jpg
463.jpg
20.jpg
301.jpg
28.jpg
61.jpg
398.jpg
349.jpg
271.jpg
286.jpg
431.jpg
316.jpg
224.jpg
411.jpg
278.jpg
456.jpg
45.jpg
324.jpg
6.jpg
59.jpg
86.jpg
470.jpg
374.jpg
132.jpg
216.jpg
95.jpg
17.jpg
162.jpg
346.jpg
439.jpg
232.jpg
150.jpg
401.jpg
145.jpg
23.jpg
514.jpg
193.jpg
416.jpg
447.jpg
220.jpg
505.jpg
147.jpg
210.jpg
40.jpg
284.jpg
55.jpg
378.jpg
520.jpg
484.jpg
112.jpg
360.jpg
317.jpg
71.jpg
202.jpg
92.jpg
11.jpg
67.jpg
138.jpg
506.jpg
146.jpg
318.jpg
198.jpg
375.jpg
41.jpg
421.jpg
129.jpg
14.jpg
460.jpg
34.jpg
380.jpg
238.jpg
392.jpg
80.jpg
517.jpg
13.jpg
189.jpg
390.jpg
487.jpg
438.jpg
482.jpg
206.jpg
217.jpg
285.jpg
385.jpg
495.jpg
269.jpg
429.jpg
291.jpg
461.jpg
256.jpg
48.jpg
290.jpg
437

In [133]:
image_list = []
for img in X_test:
  image = image_resize(img)
  image_list.append(image)
  X_test=np.array(image_list)

In [134]:
pred=model.predict(X_test)
pred=np.argmax(pred,axis=-1)

In [135]:
pred

array([2, 7, 2, 2, 1, 6, 2, 7, 1, 2, 7, 1, 5, 5, 5, 1, 2, 4, 7, 6, 2, 3,
       3, 2, 5, 3, 2, 6, 6, 1, 2, 6, 4, 6, 7, 6, 2, 2, 4, 7, 4, 7, 6, 0,
       7, 7, 7, 5, 3, 6, 7, 2, 4, 1, 4, 0, 0, 2, 6, 1, 3, 0, 4, 1, 5, 6,
       3, 6, 1, 1, 4, 1, 3, 1, 2, 3, 2, 1, 3, 4, 1, 4, 1, 2, 2, 1, 5, 4,
       0, 3, 0, 4, 4, 1, 6, 1, 5, 3, 7, 6, 1, 1, 4, 2, 7, 6, 3, 2, 0, 2,
       4, 3, 6, 2, 6, 2, 3, 3, 4, 1, 2, 6, 6, 7, 2, 7, 4, 7, 6, 2, 1, 7,
       1, 0, 0, 2, 5, 5, 1, 4, 5, 4, 0, 3, 2, 3, 2, 7, 0, 2, 7, 5, 7, 6,
       2, 5])

In [136]:
  dictionary = { 
    'manipuri' :0,
    'bharatanatyam':1,
    'odissi' :2,
    'kathakali' : 3,
    'kathak' :4,
    'sattriya':5,
    'kuchipudi':6,
    'mohiniyattam':7
}
new_dict= {value:key for key,value in dictionary.items() }

In [137]:
lst = [new_dict[i] for i in pred]

In [138]:
sr=pd.Series(lst)

In [139]:
df_test['target']=sr

In [140]:
df_test

,Image,target
0,508.jpg,odissi
1,246.jpg,mohiniyattam
2,473.jpg,odissi
3,485.jpg,odissi
4,128.jpg,bharatanatyam
...,...,...
151,366.jpg,sattriya
152,226.jpg,mohiniyattam
153,35.jpg,kuchipudi
154,458.jpg,odissi


In [141]:
df_test.to_csv('testpred6.csv', index=False)